In [1]:
import ctypes
import numpy as np

In [2]:
def init_kernel_bias(num_inp_channels, kernel_size, num_kernels,mean=0,std=0.01):
    shape = [num_inp_channels, kernel_size, kernel_size, num_kernels]
    weights = std*np.random.randn(*shape) + mean
    # weights/=np.sqrt(num_inp_channels)
    bias = std*np.random.randn(1,num_kernels) + mean
    return np.asfortranarray(weights.astype(np.float32)), np.asfortranarray(bias.astype(np.float32))

In [3]:
w0,b0=init_kernel_bias(num_inp_channels=32,kernel_size=3,num_kernels=64)

In [4]:
inp=np.arange(128*32*32*32).reshape(128,32,32,32).astype(np.float32)

In [5]:
#inp[batches,row,col,d],w0(d,ksz,ksz,num_ker),b0[1,num_ker],stride[row,col]
padding=0
stride=[1,1]
ipp=inp.transpose(0,3,1,2)  #ipp[batches,d,row,col]
output=[]
ksz=w0.shape[1]
num_ker=w0.shape[3]
if not padding: #take care of padding in backprop too
    padding=(ksz-1)//2  #currently don't give 'even' ksz
out_row,out_col=((ipp.shape[2]-ksz+2*padding)//stride[0]+1),((ipp.shape[3]-ksz+2*padding)//stride[1]+1)
batches,d,row,col=ipp.shape
row+=2*padding
col+=2*padding

In [6]:
%%time
padded=np.zeros((batches,d,row,col),dtype=np.float32)
padded[:,:,padding:-padding,padding:-padding]=ipp

CPU times: user 1.98 ms, sys: 13.4 ms, total: 15.4 ms
Wall time: 14 ms


In [7]:
# %%timeit
window=(np.arange(ksz)[:,None]*row+np.arange(ksz)).ravel()+np.arange(d)[:,None]*row*col
slider=(np.arange(out_row*stride[0])[:,None]*row+np.arange(out_col*stride[1]))
ind = window.ravel()+slider[::stride[0],::stride[1]].ravel()[:,None]
# bind= np.arange(batches)[:,None]*d*row*col+ind.ravel()
kern = w0.reshape(-1,num_ker,order='F')
# output=(np.dot(np.take(padded, bind).reshape(-1,d*ksz*ksz), kern)).reshape(batches,out_row,out_col,num_ker)

In [8]:
%%time
output=np.empty((batches,out_row*out_col,num_ker),dtype=np.float32)
for i,img in enumerate(padded):      #img[d,row,col]
    # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
    output[i]=np.dot(img.take(ind), kern)
# output+=b0
ans2=output.reshape(batches,out_row,out_col,num_ker)

CPU times: user 556 ms, sys: 16.1 ms, total: 572 ms
Wall time: 169 ms


In [9]:
%%time
coled=np.empty((batches,*ind.shape),dtype=np.float32,order='F')

CPU times: user 45 µs, sys: 0 ns, total: 45 µs
Wall time: 48.2 µs


In [10]:
img.shape

(32, 34, 34)

In [14]:
coled.shape,ind.shape,a.shape,b.shape

((128, 1024, 288), (1024, 288), (131072, 288), (288, 64))

In [ ]:
gtake.take(padded,ind,coled)     # make coled fortran style

In [469]:
(img.take(ind)==img.ravel().take(ind.ravel()).reshape(ind.shape)).all()

True

In [431]:
%%time
for i,img in enumerate(padded):      #img[d,row,col]
    # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
    coled[i]=img.take(ind)
# output+=b0
# ans2=output.reshape(batches,out_row,out_col,num_ker)
# output=(np.dot(np.take(padded, bind), self.kern)+self.biases)
# print(coled.nbytes/1024/1024)

CPU times: user 509 ms, sys: 0 ns, total: 509 ms
Wall time: 510 ms


In [432]:
np.isfortran(coled)

True

In [346]:
%%time
coled=np.asfortranarray(coled)

CPU times: user 10 µs, sys: 0 ns, total: 10 µs
Wall time: 12.4 µs


In [10]:
ind.shape

(4096, 288)

In [349]:
import concurrent

In [375]:
%%time
coled=np.empty((batches,*ind.shape),dtype=np.float32,order='F')

CPU times: user 16 µs, sys: 0 ns, total: 16 µs
Wall time: 19.1 µs


In [376]:
np.isfortran(coled)

True

In [377]:
def doit(i,img,ind):
    coled[i]=img.take(ind)

In [378]:
%%timeit
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    future_shifting = {executor.submit(doit,i,img,ind):
                         img for i,img in enumerate(padded)}

4.94 ms ± 80 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [48]:
# %%time
# for i,future in enumerate(future_shifting):
#     coled[i]=future.result()

In [49]:
np.isfortran(coled)

True

In [50]:
%%time
coled=np.asfortranarray(coled)

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 12.4 µs


In [51]:
a.shape

(524288, 288)

In [52]:
coled.shape,ind.shape,padded.shape

((128, 4096, 288), (4096, 288), (128, 32, 66, 66))

In [53]:
np.arange(16).reshape(4,4,order='F').ravel()

array([ 0,  4,  8, 12,  1,  5,  9, 13,  2,  6, 10, 14,  3,  7, 11, 15])

In [54]:
np.arange(16).reshape(4,4).ravel()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [12]:
%%time
a=coled.reshape(-1,d*ksz*ksz,order='F')
b=kern.reshape(d*ksz*ksz,-1,order='F')
(a.nbytes+b.nbytes)/1024/1024

CPU times: user 18 µs, sys: 1 µs, total: 19 µs
Wall time: 22.4 µs


In [13]:
%%time
c=np.empty((a.shape[0],b.shape[1]),dtype=np.float32,order='F')
c.nbytes/1024/1024

CPU times: user 40 µs, sys: 3 µs, total: 43 µs
Wall time: 47.9 µs


In [59]:
%%time
a=np.asfortranarray(a)
b=np.asfortranarray(b)
c=np.asfortranarray(c)

CPU times: user 321 µs, sys: 0 ns, total: 321 µs
Wall time: 219 µs


In [66]:
np.isfortran(b)

False

In [28]:
c

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [30]:
from numba import cuda

In [65]:
%%time
A = cuda.to_device(a)
B = cuda.to_device(b)
C = cuda.to_device(c)

CPU times: user 401 ms, sys: 121 ms, total: 521 ms
Wall time: 520 ms


In [69]:
A

In [32]:
sgemm=ctypes.CDLL('libsgemm.so')

In [52]:
al,bet=1,0

In [53]:
A.device_ctypes_pointer

c_ulong(21490696192)

In [70]:
%%time
sgemm.gemm(A.device_ctypes_pointer,B.device_ctypes_pointer,C.device_ctypes_pointer,a.shape[0],b.shape[0],c.shape[1],ctypes.c_float(al),ctypes.c_float(bet),ctypes.c_void_p(b0[0].ctypes.data))

CPU times: user 94.7 ms, sys: 40.6 ms, total: 135 ms
Wall time: 131 ms


0

In [68]:
# %%time
# sgemm.gemm(ctypes.c_void_p(a.ctypes.data),ctypes.c_void_p(b.ctypes.data),ctypes.c_void_p(c.ctypes.data),a.shape[0],b.shape[0],c.shape[1],ctypes.c_float(al),ctypes.c_float(bet),ctypes.c_void_p(b0[0].ctypes.data))

CPU times: user 483 ms, sys: 200 ms, total: 683 ms
Wall time: 682 ms


0

In [69]:
a.shape[0],b.shape[0],c.shape[1]

(524288, 288, 128)

In [70]:
# al*np.dot(a,b)+bet*c

In [58]:
c

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [59]:
ans2.reshape(-1,128)

array([[-0.05595898, -0.21922275, -0.02473635, ...,  0.00049529,
        -0.06479301,  0.12042055],
       [-0.29891318,  0.06180175, -0.22781079, ..., -0.07143477,
        -0.17832406, -0.07867721],
       [-0.00855565,  0.16976127,  0.13988197, ...,  0.08600792,
        -0.09984475,  0.00938692],
       ...,
       [-0.17049578, -0.02128632,  0.04620727, ...,  0.08487362,
        -0.07393762, -0.10049743],
       [ 0.1636887 , -0.05334401, -0.23741074, ..., -0.03989929,
         0.01319493,  0.02640355],
       [ 0.10810144,  0.05109768,  0.22799608, ...,  0.06546371,
         0.040252  , -0.08171172]], dtype=float32)

In [57]:
C.copy_to_host()

array([[-0.05595898, -0.21922275, -0.02473635, ...,  0.00049529,
        -0.06479301,  0.12042055],
       [-0.29891318,  0.06180175, -0.22781079, ..., -0.07143477,
        -0.17832406, -0.07867721],
       [-0.00855565,  0.16976127,  0.13988197, ...,  0.08600792,
        -0.09984475,  0.00938692],
       ...,
       [-0.17049578, -0.02128632,  0.04620727, ...,  0.08487362,
        -0.07393762, -0.10049743],
       [ 0.1636887 , -0.05334401, -0.23741074, ..., -0.03989929,
         0.01319493,  0.02640355],
       [ 0.10810144,  0.05109768,  0.22799608, ...,  0.06546371,
         0.040252  , -0.08171172]], dtype=float32)

In [73]:
(c==ans2.reshape(-1,128)).all()

True

In [23]:
# a=(np.arange(30)+11).reshape(6,5,order='F').astype(np.float32)  #mxk

In [24]:
# b=(np.arange(20)+11).reshape(5,4,order='F').astype(np.float32)  #kxn

In [25]:
# c=(np.arange(24)*0).reshape(6,4,order='F').astype(np.float32)  #mxn